In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import tqdm

import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
from scipy import exp
from scipy.optimize import curve_fit

import pandas as pd
import os
import shutil
import cv2

import glob2

from numba import jit
import tables
from scipy.io import loadmat
import scipy
import h5py
import hdf5storage
from sklearn.metrics.pairwise import euclidean_distances
import matplotlib.cm as cm

from utility_simulated_drift import (make_shifted_templates,
                                      make_default_template,
                                      draw_template,
                                      gaus,
                                      make_drift_template,
                                      shift_templates_all,
                                      WaveForms,
                                      generate_poisson_uniform_firingrate2,
                                      select_ground_truth_units,
                                      visualize_drift,
                                      generate_synthetic_data,
                                      visualize_traces,
                                      make_full_data)

colors = [
'black','grey','brown','slategrey',    
'orange','firebrick','lawngreen','dodgerblue','crimson','orchid','slateblue',
'darkgreen','darkorange','indianred','darkviolet','deepskyblue','greenyellow',
'peru','cadetblue','forestgreen','slategrey','lightsteelblue','rebeccapurple',
'darkmagenta','yellow','hotpink']
clrs= colors


Autosaving every 180 seconds


/home/cat/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# select working directory location
root_dir = '/media/cat/12TB/insync_cm3746/paninski_lab/data/neuropixels/synthetic_drift/run15_30mins_40umDrift/'
try:
    os.mkdir(root_dir)
except:
    pass

# set radius of channels to consider from maximum channel outwards (in micrometers)
radius = 200

# select ground truth templates to be used
fname_templates = '/media/cat/12TB/dbox/Dropbox/code/neuropixels/allen_inst_2ndrec/templates_reloaded.npy'
geom = np.loadtxt('/home/cat/p1_g0_t0.imec0.ap_geom_384chans.txt')
#fname_templates = '/media/cat/1TB/data/synthetic/allen_inst_2ndrec/templates_reloaded.npy'
#geom = np.loadtxt('/media/cat/1TB/data/synthetic/p1_g0_t0.imec0.ap_geom.txt')



# select largest X templates to be used for injection
n_templates = 100
(units, templates_ground_truth) = select_ground_truth_units(root_dir, 
                                                            fname_templates,
                                                            n_templates,
                                                            geom)

print ("templates ground truth: ", templates_ground_truth.shape)

Total templates avialable;  (575, 101, 384)
Total units selected:  100 , ids:  [420 339 421 348 473 418  62 362 565 239] ...
templates ground truth:  (100, 101, 384)


In [4]:
# visualize the desired shifts on a particular unit
shifts = np.arange(0, .05, 0.05)

# visualize drifted template for any particular unit 
unit_id = 10
visualize_drift(shifts, geom, templates_ground_truth, unit_id,
               radius)


In [3]:
# generate synthetic drift data
sample_rate = 30000
rec_len_sec = 1800
rec_length_sample_times = rec_len_sec * sample_rate

# spikes, scales = generate_poisson_uniform_firingrate2(rec_length)
# setup shifts
shifts = np.arange(0, 1.10, 0.05)
print ("shifts: ", shifts)

data_synthetic = generate_synthetic_data(root_dir,
                                         rec_len_sec, 
                                         sample_rate, 
                                         shifts, 
                                         units, 
                                         templates_ground_truth,
                                         radius,
                                         geom)

np.savetxt(root_dir+'/ground_truth/shifts.txt', shifts)
print (data_synthetic.shape)


shifts:  [0.   0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65
 0.7  0.75 0.8  0.85 0.9  0.95 1.   1.05]
(54000000, 384)
[       0.          2571428.57142857  5142857.14285714  7714285.71428571
 10285714.28571429 12857142.85714286 15428571.42857143 18000000.
 20571428.57142857 23142857.14285714 25714285.71428571 28285714.28571428
 30857142.85714286 33428571.42857143 36000000.         38571428.57142857
 41142857.14285714 43714285.71428572 46285714.28571428 48857142.85714286
 51428571.42857143 54000000.        ]
window:  [0.0, 2571428.5714285714] , shift:  0.0  (inter channel units)
 inserting unit:  0 [ 6512 13777 23794 28643 37630 38273 41064 43514 49254 56030] [1.         1.         1.         1.         1.         0.92403992
 0.99302444 0.98216103 1.         1.        ]
 inserting unit:  50 [ 2694  3956  5317  9982 19715 38161 38936 39573 52672 60681] [1.         0.94364995 0.94743211 1.         1.         1.
 0.92774349 0.92496443 1.         1.        ]
************

 inserting unit:  50 [36005515 36022732 36039459 36045263 36047982 36068632 36068801 36086856
 36106606 36106981] [1.         1.         1.         1.         0.99104038 1.
 0.91028276 1.         1.         0.9166771 ]
************


window:  [38571428.57142857, 41142857.14285714] , shift:  0.75  (inter channel units)
 inserting unit:  0 [38571851 38571969 38577018 38577071 38577647 38579363 38581762 38585764
 38586758 38587014] [0.9351952  0.90846402 1.         0.9066489  0.92219369 0.95791139
 0.98019867 1.         0.9351952  0.91210515]
 inserting unit:  50 [38585749 38586471 38600304 38603990 38610727 38611110 38630500 38638210
 38643163 38645182] [1.         0.92681621 1.         1.         1.         0.9166771
 1.         1.         1.         0.96753856]
************


window:  [41142857.14285714, 43714285.71428572] , shift:  0.8  (inter channel units)
 inserting unit:  0 [41142925 41145568 41149162 41149533 41154305 41165162 41167055 41175289
 41178387 41185917] [1.         0.9

In [6]:
# visualize the final traces
visualize_traces(data_synthetic, 0, 5000)


In [4]:
# Make a correlated data chunk of 10 seconds 
# longer takes too long of time; shorter will add more boundary artifcats to recording
time = 10*sample_rate
print ("length of correlated noise snipit: ", time)

fname = root_dir+'correlated_noise.npy'
fname = '/media/cat/4TBSSD/liam/neuropixels/synthetic/correlated_noise.npy'
# use ground_truth_templates to make noise
if os.path.exists(fname)==False:
    temps = templates_ground_truth.copy()
    WF = WaveForms(temps.transpose(0,2,1))
    correlated_noise = WF.generate_correlated_noise_add(time)
    print ("DONE MAKING CORRELATED NOISE for 10 sec chunk")
    
    np.save(fname, correlated_noise)
else:
    correlated_noise = np.load(fname)
print ("DONE generating correlated noise")

length of correlated noise snipit:  300000
DONE generating correlated noise


In [5]:
# blend correlated noise with injected spike dataset
data_sum = make_full_data(data_synthetic, rec_length_sample_times, 
                          sample_rate, root_dir,
                          correlated_noise)


white noise:  (300000, 384)
time chunk:  0  to  300000
time chunk:  300000  to  600000
time chunk:  600000  to  900000
time chunk:  900000  to  1200000
time chunk:  1200000  to  1500000
time chunk:  1500000  to  1800000
time chunk:  1800000  to  2100000
time chunk:  2100000  to  2400000
time chunk:  2400000  to  2700000
time chunk:  2700000  to  3000000
time chunk:  3000000  to  3300000
time chunk:  3300000  to  3600000
time chunk:  3600000  to  3900000
time chunk:  3900000  to  4200000
time chunk:  4200000  to  4500000
time chunk:  4500000  to  4800000
time chunk:  4800000  to  5100000
time chunk:  5100000  to  5400000
time chunk:  5400000  to  5700000
time chunk:  5700000  to  6000000
time chunk:  6000000  to  6300000
time chunk:  6300000  to  6600000
time chunk:  6600000  to  6900000
time chunk:  6900000  to  7200000
time chunk:  7200000  to  7500000
time chunk:  7500000  to  7800000
time chunk:  7800000  to  8100000
time chunk:  8100000  to  8400000
time chunk:  8400000  to  870000

In [6]:
# visualize first 10000 time points;
visualize_traces(data_sum, 0, 10000)


In [ ]:
DOONE

In [7]:
# visualize spike triggered max chan data
spike_train_gt = np.load('/media/cat/1TB/data/synthetic/run11/ground_truth/spike_train_ground_truth.npy')
fname= '/media/cat/1TB/data/synthetic/run11/data_int16.bin'

from utility_match_units import binary_reader_waveforms

In [14]:
n_channels = 384
n_times = 101

# 
units = np.arange(25)
units = [13]
for unit in units:
    #ax=plt.subplot(5,5,unit+1)
    idx = np.where(spike_train_gt[:,1]==unit)[0]
    spikes = spike_train_gt[idx,0]
    wfs = binary_reader_waveforms(fname, n_channels, n_times, spikes, data_type='int16')

    print (wfs.shape)
    max_chan = wfs.mean(0).ptp(0).argmax(0)

    #fig=plt.figure()
    plt.plot(wfs[:,:,max_chan].T,c='black')
    plt.show()

    if True:
        fig=plt.figure()
        for k in range(wfs.shape[0]):
            ax=plt.subplot(10,10,k+1)
            plt.plot(wfs[k,:,max_chan].T,c='black')
            plt.title(str(k)+" "+str(spikes[k]))
        plt.show()

(85, 101, 384)


In [15]:
times = np.load('/media/cat/1TB/temp/'+str(unit)+'_times.npy')
scale = np.load('/media/cat/1TB/temp/'+str(unit)+'_scale.npy')

print (times.shape)
print (scale.shape)
print (scale)


(85,)
(85,)
[1.         0.69767633 0.64403642 1.         1.         0.99501248
 0.74453159 0.71177032 0.68045064 1.         1.         0.89136614
 0.79057085 1.         1.         1.         0.68728928 1.
 0.81873075 1.         1.         0.93706746 0.86070798 0.83945702
 1.         0.97044553 0.72614904 0.86935824 0.85641518 0.98019867
 1.         0.66034028 1.         0.62188506 1.         0.99004983
 0.69073433 0.77105159 0.83110428 0.70117344 1.         0.99004983
 1.         0.88692044 0.81058425 1.         0.61878339 0.92774349
 1.         0.63762815 1.         0.6156972  1.         1.
 0.68728928 0.78662786 1.         0.60957091 0.60957091 0.77880078
 0.85214379 1.         1.         1.         1.         0.72614904
 0.70117344 1.         1.         1.         0.66697681 0.99501248
 1.         1.         0.62188506 0.81873075 0.62500227 0.66034028
 0.65376979 1.         0.82695913 0.64082428 0.64082428 1.
 1.        ]
